In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from win32com.client import DispatchEx
import os
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
# import pdfkit
# import jpype
# import asposecells
# from asposecells.api import Workbook, FileFormatType, PdfSaveOptions
#import xlsxwriter



In [5]:
import string
alph = string.ascii_lowercase.upper()
#creat PDF file
def to_pdf(df2, path=None, filename="table"):

    if path == None:
        return
    wb = load_workbook(path)
    ws = wb.active
    
    dims = {}
    for row in ws.rows:
        for cell in row:
            if cell.value:
                dims[cell.column] = max((dims.get(cell.column, 0), len(str(cell.value))))    
    for col, value in dims.items():
        print(col)
        ws.column_dimensions[alph[col-1]].width = value
    wb.save(filename=path)
#     column_widths = []
#     for row in data:
#         for i, cell in enumerate(row):
#             if len(column_widths) > i:
#                 if len(cell) > column_widths[i]:
#                     column_widths[i] = len(cell)
#             else:
#                 column_widths += [len(cell)]

#     for i, column_width in enumerate(column_widths,1):  # ,1 to start at 1
#         worksheet.column_dimensions[get_column_letter(i)].width = column_width
    
    
#     # Create and set PDF options
#     pdfOptions = PdfSaveOptions()
#     pdfOptions.setCompliance(PdfCompliance.PDF_A_1_B)

#     # Convert Excel to PDF
#     workbook.save(r'C:\Users\明\Desktop\BMW\{}.pdf'.format(filename), pdfOptions)
    excel_path = path
    pdf_path = r'U:\BMW\{}.pdf'

    df = DispatchEx("Excel.Application")
    df.Visible = False
    df.DisplayAlerts = 0
    books = df.Workbooks.Open(excel_path,False)
    books.ExportAsFixedFormat(0, pdf_path.format(filename))
    books.Close(False)
    df.Quit()


In [6]:
filename = r'U:\BMW\2022-04-04_10-16_beat_softwareverbund_I_K22_22-07_505_22w13_DE61.xlsx'
sheetname = 'Funkt_und_Bewertung_n_Standort'

In [7]:
# DICTIONARY TO CLASSIFY GROUP BASED ON KEYWORDS
# keyword : GROUP

KEYWORDS = {
    #NAV
    "Display map ASIA" : "NAV",
    "POI Functions ASIA" : "NAV",
    "Manage and Plan Routes ASIA" : "NAV",
    "Navigation Destination Input ASIA" : "NAV",
    "Map and Navigation Data update ASIA" : "NAV",
    "Display map ASIA" : "NAV",
    "Traffic Info ASIA" : "NAV",
    "Guiding ASIA" : "NAV",
    "Positioning ASIA" : "NAV",
    "Guiding ASIA" : "NAV",
    "" : "NAV",
    "" : "NAV",
    #ENT
    "CarPlay" : "ENT",
    "Play audio via USB" : "ENT",
    "Display & operation in the rear of entertainment sources (BMT)" : "ENT",
    "Manage CE Devices (Bluetooth)" : "ENT",
    "Play audio via Bluetooth" : "ENT",
    "elephony via second customer device" : "ENT",
    "elephony via customer device" : "ENT",
    "Watch TV (Japan)" : "ENT",
    "Listen to AM/FM Radio (Legacy < IDC Android)" : "ENT",
    "Show Call List" : "ENT",
    "Display Contacts" : "ENT",
    "Search Contacts" : "ENT",
    "FireTV download of content" : "ENT",
    "Consumer e-SIM Management" : "ENT",
    "Display and operation of PaDi BMW widgets" : "ENT",
    "FireTV Full App Support and Automotive Feature Set" : "ENT",
    "FireTV VoD App and Audio Output" : "ENT",
    "Internet Access via Hotspot" : "ENT",
    "PaDi Remote Control via BMT" : "ENT",
    "PaDi RoW CE area (FireTV) bring-up" : "ENT",
    "PaDi View and Audio Settings" : "ENT",
    "Handsfree Telephony in the rear seats" : "ENT",
    "Telephony via customer SIM" : "ENT",
    "Display map ASIA" : "ENT",
    "" : "ENT",
    "" : "ENT",
    "" : "ENT",
    "" : "ENT",
    "" : "ENT",
    "" : "ENT",
    "" : "ENT",
    
    #HMI
    "Display of User Manual" : "HMI",
    "ZBE Control" : "HMI",
    "Augmented Reality Video CID" : "HMI",
    "" : "HMI",
    "" : "HMI",
    "" : "HMI",
    "" : "HMI",
    "" : "HMI",
    #DIPS
    " E-Call" : "DIPS",
    "Call Customer service" : "DIPS",
    "Concierge Call" : "DIPS",
    "Voice Interface" : "DIPS",
    "OAP App platform Vehicle-App-Interface Setup" : "DIPS",
    "Remote Services" : "DIPS",
    "Report service issues via Teleservice" : "DIPS",
    "Transfer Data from Vehicle to Backend (Last State Call)" : "DIPS",
#     "" : "DIPS",
#     "" : "DIPS",
#     "" : "DIPS",
#     "" : "DIPS",
#     "" : "DIPS",
    
}

def group(x):
    for g in KEYWORDS.keys():
        if g in x:
            return KEYWORDS[g]
    return "Default"

In [18]:
#edit content
def remove_time(x):
    y = x.split("\n")
    if ":" in y[0]:
        y[0] = ":".join(y[0].split(':')[2:])
    elif " " in y[0]:
        y[0] = " ".join(y[0].split(' ')[2:])
    
    return "\n".join(y)

def remove_MLI(x):
    y = x.split("\n")
    for i in y:
        if i == '[MLI]':
           MLI_index = y.index('[MLI]') 
           if y[MLI_index + 1] == '':
#                y.remove (y[MLI_index])
#                y.remove (y[MLI_index+1])
                #y = (y[MLI_index:])
                y = (y[MLI_index+3:])
        if i == '–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––                      ':
          y.remove(i)
    return y

def remove_TOP(x):
    y = x.split("\n")
    for i in y:
        if i == '[Top5 defects]':
           MLI_index = y.index('[MLI]') 
           if y[MLI_index + 1] == '':
#                y.remove (y[MLI_index])
#                y.remove (y[MLI_index+1])
                #y = (y[MLI_index:])
                y = (y[MLI_index+3:])
# def remove_top(x):
#     y = x.split("\n")
#     top_index = y.index('[Top5 defects]')
#     if y[top_index + 1] == '':
#        y.remove (y[top_index])
#        y.remove (y[top_index+1])
#     return y

#  def remove_words(x):
#     ticket = re.sub('[MLI]\n\n', x)

In [19]:
# creat new Excel colum
data = pd.read_excel(filename,sheet_name=sheetname) 
# X = pd.DataFrame(data, columns= ['Standort Name'])
# print (df)

df = pd.DataFrame(data, columns= ['Name EN Reportelement','AIDA IP RG','Standort RG','Standort Bemerkung','Standort Name'])
df = df[df['Standort Name'] == 'JAP']
groups = [group(x) for x in df['Name EN Reportelement']]
df['Standort Bemerkung'] = [remove_time(x) for x in df['Standort Bemerkung']]
print(df['Standort Bemerkung'])
df['Standort Bemerkung'] = [remove_MLI(x) for x in df['Standort Bemerkung']]
#df['Standort Bemerkung'] = [remove_top(x) for x in df['Standort Bemerkung']]
cols = df.columns.tolist()
cols = ["Group"] + cols
df["Group"] = groups
df = df[cols]
print (df)
patth = r'U:\BMW\listv4.xlsx'
df.to_excel(patth)
#to_pdf(df, filename="list7")


6      \n\n[MLI]\n\n[Top5 defects]\n\n–––––––––––––––...
7      \n\n[MLI]\n\n\n[Top5 defects]\n\n–––––––––––––...
15     \n\n[MLI]\n\n\n[Top5 defects]\n904258 [JP][MGU...
24     \n\n[MLI]\n\n[Top5 defects]\n\n\n\n–––––––––––...
30     \n\n[MLI]\n\n[Top5 defects]\n\n\n–––––––––––––...
                             ...                        
405    \n\n[MLI]\n\n[Top5 defects]\n\n\n[Device used ...
410    \n\n[MLI]\n\n\n[Top5 defects]\n877250 [MGU22][...
416    \n\n[MLI]\n\n[Top5 defects]\n896121 [CW07][JP]...
428    \n\n[MLI]\n\n[Top5 defects]\n938980 [JP][speec...
443    \n\n[MLI]\n\n[Top5 defects]\n945208 [JP][MGU22...
Name: Standort Bemerkung, Length: 70, dtype: object
    Group                              Name EN Reportelement AIDA IP RG  \
6     HMI                                          ADAS ASIA        5-6   
7     HMI             Acoustic request to audio control unit        5-6   
15    HMI      Listen to AM/FM Radio  (Legacy < IDC Android)        5-6   
24    HMI            

In [3]:
#to_pdf(df,path = patth, filename="listv1pdf")